# 🧪 Codelab: Grover's Applications & Extensions

| Metadata | Value |
|----------|-------|
| **Algorithm** | Quantum Counting, Amplitude Estimation, Adaptive Search |
| **Difficulty** | 🟡 Intermediate |
| **Time** | 90-120 minutes |
| **Prerequisites** | Grover's Search (Module 7.7), QPE basics (Module 7.5) |
| **Qiskit Version** | 2.x |

---

## Learning Objectives

By the end of this codelab, you will be able to:

1. ✅ Implement quantum counting to find the number of solutions
2. ✅ Build amplitude estimation circuits
3. ✅ Use adaptive search when M is unknown
4. ✅ Construct oracles for SAT problems
5. ✅ Analyze the precision of counting algorithms

## Section 1: Environment Setup & Version Check

In [ ]:
# Required imports
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Optional, Tuple, Union, Callable
from math import floor, sqrt, pi, ceil
import random

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.primitives import StatevectorSampler as SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.compiler import transpile
from qiskit.circuit.library import QFT

# Version check - Qiskit 2.x required
import qiskit
version = qiskit.__version__
major_version = int(version.split('.')[0])
assert major_version >= 1, f"Qiskit 2.x required, found {version}"
print(f"✓ Qiskit version: {version}")
print(f"✓ NumPy version: {np.__version__}")
print("✓ All imports successful!")

## Section 2: Theory Recap

### Beyond Simple Search

Grover's algorithm answers: **"Is there a solution?"**

Grover's extensions answer:
- **Quantum Counting**: "How many solutions exist?" → $O(\sqrt{N})$
- **Amplitude Estimation**: "What's the probability of success?" → $O(1/\epsilon)$
- **Adaptive Search**: "Find a solution when M is unknown" → $O(\sqrt{N/M})$

### Key Insight: Grover as a Rotation

The Grover operator $G = D_s \cdot O_f$ rotates by angle $2\theta$ where:

$$\sin^2(\theta) = \frac{M}{N}$$

- $M$ = number of solutions
- $N$ = size of search space

**Eigenvalues of G**: $e^{\pm 2i\theta}$

If we can measure $\theta$ using QPE, we can compute $M$!

## Section 3: Building Blocks from Grover

First, let's implement the core Grover components we'll reuse.

In [ ]:
def create_oracle(n_qubits: int, targets: Union[int, str, List]) -> QuantumCircuit:
    """
    Create a phase oracle that marks target state(s) with a phase flip.
    
    Args:
        n_qubits: Number of qubits
        targets: Target state(s) as integer, bitstring, or list
    
    Returns:
        Oracle circuit
    """
    qc = QuantumCircuit(n_qubits, name='Oracle')
    
    # Convert targets to list
    if not isinstance(targets, list):
        targets = [targets]
    
    for t in targets:
        # Convert to binary string if integer
        if isinstance(t, int):
            t = format(t, f'0{n_qubits}b')
        
        # Apply X gates to qubits that should be |0⟩
        for i, bit in enumerate(reversed(t)):
            if bit == '0':
                qc.x(i)
        
        # Multi-controlled Z
        if n_qubits == 1:
            qc.z(0)
        elif n_qubits == 2:
            qc.cz(0, 1)
        else:
            qc.h(n_qubits - 1)
            qc.mcx(list(range(n_qubits - 1)), n_qubits - 1)
            qc.h(n_qubits - 1)
        
        # Undo X gates
        for i, bit in enumerate(reversed(t)):
            if bit == '0':
                qc.x(i)
    
    return qc


def create_diffuser(n_qubits: int) -> QuantumCircuit:
    """
    Create the Grover diffuser: D_s = 2|s⟩⟨s| - I
    """
    qc = QuantumCircuit(n_qubits, name='Diffuser')
    qc.h(range(n_qubits))
    qc.x(range(n_qubits))
    
    if n_qubits == 1:
        qc.z(0)
    elif n_qubits == 2:
        qc.cz(0, 1)
    else:
        qc.h(n_qubits - 1)
        qc.mcx(list(range(n_qubits - 1)), n_qubits - 1)
        qc.h(n_qubits - 1)
    
    qc.x(range(n_qubits))
    qc.h(range(n_qubits))
    
    return qc


def create_grover_operator(n_qubits: int, oracle: QuantumCircuit) -> QuantumCircuit:
    """
    Create the full Grover operator G = D_s · O_f
    """
    qc = QuantumCircuit(n_qubits, name='G')
    qc.compose(oracle, inplace=True)
    qc.compose(create_diffuser(n_qubits), inplace=True)
    return qc


# Test the building blocks
n = 3
targets = [0, 7]  # |000⟩ and |111⟩
oracle = create_oracle(n, targets)
grover_op = create_grover_operator(n, oracle)

print("Grover Operator for 3 qubits with M=2 solutions:")
print(grover_op.draw())

## Section 4: Quantum Counting

### The Key Idea

The Grover operator $G$ has eigenvalues $e^{\pm 2i\theta}$ where $\sin^2(\theta) = M/N$.

**Strategy**: Use QPE to estimate $\theta$, then compute $M = N \cdot \sin^2(\theta)$.

### Circuit Structure

```
|0⟩^⊗t ──H──■──────■────────■──────────[QFT†]──Measure
            │      │        │
|0⟩^⊗n ──H──G──────G²───────G^{2^{t-1}}
```

In [ ]:
def controlled_grover_power(n_search: int, oracle: QuantumCircuit, power: int) -> QuantumCircuit:
    """
    Create controlled-G^power circuit.
    
    Args:
        n_search: Number of search qubits
        oracle: The oracle circuit
        power: The power of G to apply
    
    Returns:
        Controlled circuit with 1 control + n_search target qubits
    """
    # Build G first
    G = create_grover_operator(n_search, oracle)
    
    # Create controlled version
    c_G = G.control(1, label=f'c-G^{power}')
    
    # Build circuit applying G power times
    qc = QuantumCircuit(1 + n_search, name=f'c-G^{power}')
    for _ in range(power):
        qc.append(c_G, range(1 + n_search))
    
    return qc


def quantum_counting_circuit(n_count: int, n_search: int, oracle: QuantumCircuit) -> QuantumCircuit:
    """
    Build the full quantum counting circuit.
    
    Args:
        n_count: Number of counting qubits (precision)
        n_search: Number of search qubits (N = 2^n_search)
        oracle: Oracle marking solutions
    
    Returns:
        Quantum counting circuit
    """
    # Registers
    count_reg = QuantumRegister(n_count, 'count')
    search_reg = QuantumRegister(n_search, 'search')
    classical_reg = ClassicalRegister(n_count, 'c')
    
    qc = QuantumCircuit(count_reg, search_reg, classical_reg)
    
    # Initialize counting register in superposition
    qc.h(count_reg)
    
    # Initialize search register in uniform superposition
    qc.h(search_reg)
    
    qc.barrier()
    
    # Apply controlled-G^(2^j) for each counting qubit
    for j in range(n_count):
        power = 2**j
        c_G_power = controlled_grover_power(n_search, oracle, power)
        qc.append(c_G_power, [count_reg[j]] + list(search_reg))
    
    qc.barrier()
    
    # Inverse QFT on counting register
    qft_dagger = QFT(n_count, inverse=True, do_swaps=True)
    qc.append(qft_dagger, count_reg)
    
    # Measure counting register
    qc.measure(count_reg, classical_reg)
    
    return qc


# Build a small example
n_count = 4  # 4 bits of precision
n_search = 3  # N = 8 items
M_true = 2    # 2 solutions: |000⟩ and |111⟩

oracle = create_oracle(n_search, [0, 7])
qc_counting = quantum_counting_circuit(n_count, n_search, oracle)

print(f"Quantum Counting Circuit (t={n_count} precision qubits, n={n_search} search qubits):")
print(f"Looking for M={M_true} solutions among N={2**n_search} items\n")
print(qc_counting.draw(fold=100))

In [ ]:
def extract_M_from_measurement(measured: int, n_count: int, N: int) -> Tuple[float, float]:
    """
    Convert QPE measurement to estimated number of solutions.
    
    The eigenvalue is e^{2iθ} where sin²(θ) = M/N.
    QPE measures θ̃ ≈ θ/π * 2^t or (2π - θ)/π * 2^t
    
    Args:
        measured: Integer from counting register measurement
        n_count: Number of counting qubits
        N: Size of search space
    
    Returns:
        (M_estimate, theta_estimate)
    """
    # Convert measurement to phase
    theta = np.pi * measured / (2**n_count)
    
    # Extract M from sin²(θ) = M/N
    M_estimate = N * np.sin(theta)**2
    
    return M_estimate, theta


# Run the quantum counting circuit (decompose to handle custom gates)
# Decompose multiple times to ensure all custom gates are broken down
qc_counting_decomposed = qc_counting.decompose().decompose().decompose()
shots = 1024

# Execute with SamplerV2 (no backend needed for statevector simulation)
sampler = SamplerV2()
job = sampler.run([qc_counting_decomposed], shots=shots)
result = job.result()
counts = result[0].data.c.get_counts()

print("Measurement Results:")
print("=" * 50)

# Analyze results
N = 2**n_search
theta_true = np.arcsin(np.sqrt(M_true / N))
print(f"\nTrue values: M = {M_true}, θ = {theta_true:.4f} rad = {np.degrees(theta_true):.2f}°")
print(f"\nMeasurement → Estimated M:")

estimates = []
for bitstring, count in sorted(counts.items(), key=lambda x: -x[1])[:5]:
    measured = int(bitstring, 2)
    M_est, theta_est = extract_M_from_measurement(measured, n_count, N)
    estimates.extend([M_est] * count)
    print(f"  |{bitstring}⟩ = {measured:3d}: M ≈ {M_est:.2f}, θ ≈ {np.degrees(theta_est):.2f}° (count: {count})")

print(f"\nMean estimated M: {np.mean(estimates):.2f} (True M = {M_true})")


In [ ]:
# Visualize the results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Measurement histogram
ax1 = axes[0]
sorted_counts = dict(sorted(counts.items(), key=lambda x: int(x[0], 2)))
ax1.bar(range(len(sorted_counts)), sorted_counts.values(), tick_label=list(sorted_counts.keys()))
ax1.set_xlabel('Measured bitstring')
ax1.set_ylabel('Counts')
ax1.set_title('Quantum Counting Measurement Results')
ax1.tick_params(axis='x', rotation=45)

# Plot 2: Phase circle
ax2 = axes[1]
theta = np.linspace(0, 2*np.pi, 100)
ax2.plot(np.cos(theta), np.sin(theta), 'b-', alpha=0.3)

# Mark true eigenvalue positions
theta_true = np.arcsin(np.sqrt(M_true / N))
ax2.plot(np.cos(2*theta_true), np.sin(2*theta_true), 'go', markersize=15, label=f'e^{{2iθ}}, θ={np.degrees(theta_true):.1f}°')
ax2.plot(np.cos(-2*theta_true), np.sin(-2*theta_true), 'ro', markersize=15, label=f'e^{{-2iθ}}')

# Mark measured phases
for bitstring in list(counts.keys())[:3]:
    measured = int(bitstring, 2)
    theta_meas = np.pi * measured / (2**n_count)
    ax2.plot(np.cos(2*theta_meas), np.sin(2*theta_meas), 'x', markersize=10, 
             label=f'Measured: {bitstring}')

ax2.set_xlim(-1.3, 1.3)
ax2.set_ylim(-1.3, 1.3)
ax2.set_aspect('equal')
ax2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
ax2.axvline(x=0, color='k', linestyle='-', alpha=0.3)
ax2.set_title('Eigenvalue Phase Circle')
ax2.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()

## Section 5: State Evolution Analysis

### 5.5.1 Key Identity: Eigenvalue-to-Count Connection

**Rule Name**: "Eigenphase Encodes Solution Count"

The Grover operator $G$ has eigenvalues that encode the number of solutions:

$$\boxed{G|\psi_\pm\rangle = e^{\pm 2i\theta}|\psi_\pm\rangle \quad \text{where} \quad M = N\sin^2(\theta)}$$

**Why This Works**:
1. The initial state $|s\rangle = \sin(\theta)|w\rangle + \cos(\theta)|s'\rangle$ decomposes into $G$'s eigenstates
2. QPE measures the eigenphase $\pm 2\theta$
3. Inverting $\sin^2(\theta) = M/N$ gives $M$

**Key Insight**: The structure of $G$ (determined by $M$) is encoded in its spectrum, not just its action!

### 5.5.2 State Evolution Table: Quantum Counting

| Stage | Count Register | Search Register | Mathematical State |
|-------|---------------|-----------------|--------------------|
| **Initial** | $\|0\rangle^{\otimes t}$ | $\|0\rangle^{\otimes n}$ | $\|0\rangle^{\otimes t} \otimes \|0\rangle^{\otimes n}$ |
| **After H** | $\|+\rangle^{\otimes t}$ | $\|s\rangle = H^{\otimes n}\|0\rangle$ | $\frac{1}{\sqrt{2^t}}\sum_{j=0}^{2^t-1}\|j\rangle \otimes \|s\rangle$ |
| **After controlled-G** | Phase encoded | $G^j\|s\rangle$ accumulated | $\frac{1}{\sqrt{2^t}}\sum_j\|j\rangle \otimes G^j\|s\rangle$ |
| **Eigenstate decomp.** | Phases $\pm 2j\theta$ | $\|\psi_+\rangle, \|\psi_-\rangle$ | Sum over eigenstates with phase $e^{\pm 2ij\theta}$ |
| **After QFT$^\dagger$** | $\|\tilde{\theta}\rangle$ | Unchanged | Measured phase encodes $\theta$ |
| **After measurement** | Classical $\tilde{\theta}$ | Collapsed | $M \approx N\sin^2(\pi\tilde{\theta}/2^t)$ |

In [ ]:
def trace_counting_state(n_count: int, n_search: int, oracle: QuantumCircuit, 
                         M_true: int) -> dict:
    """
    Trace the quantum state through key stages of quantum counting.
    
    Args:
        n_count: Number of counting qubits
        n_search: Number of search qubits
        oracle: Oracle circuit
        M_true: True number of solutions (for verification)
    
    Returns:
        Dictionary with state information at each stage
    """
    N = 2**n_search
    theta_true = np.arcsin(np.sqrt(M_true / N))
    
    trace_info = {
        'N': N,
        'M_true': M_true,
        'theta_true': theta_true,
        'stages': []
    }
    
    # Build circuit incrementally
    count_reg = QuantumRegister(n_count, 'count')
    search_reg = QuantumRegister(n_search, 'search')
    qc = QuantumCircuit(count_reg, search_reg)
    
    # Stage 1: Initial state
    sv = Statevector(qc)
    trace_info['stages'].append({
        'name': 'Initial',
        'description': '|0⟩^⊗(t+n)',
        'state_info': f'All qubits in |0⟩',
        'key_amplitude': f'|0...0⟩: {abs(sv.data[0]):.4f}'
    })
    
    # Stage 2: After H on all qubits
    qc.h(count_reg)
    qc.h(search_reg)
    sv = Statevector(qc)
    uniform_amp = 1/np.sqrt(2**(n_count + n_search))
    trace_info['stages'].append({
        'name': 'After Hadamard',
        'description': '|+⟩^⊗t ⊗ |s⟩',
        'state_info': 'Uniform superposition',
        'key_amplitude': f'All amplitudes: {uniform_amp:.4f}'
    })
    
    # Stage 3: After controlled-G operations
    G = create_grover_operator(n_search, oracle)
    for j in range(n_count):
        power = 2**j
        c_G = G.control(1)
        for _ in range(power):
            qc.append(c_G, [count_reg[j]] + list(search_reg))
    
    sv = Statevector(qc)
    # Find the most significant amplitudes
    amp_data = np.abs(sv.data)
    max_idx = np.argmax(amp_data)
    trace_info['stages'].append({
        'name': 'After controlled-G powers',
        'description': 'Phase information encoded',
        'state_info': f'Eigenphases ±2jθ accumulated',
        'key_amplitude': f'Max amplitude at index {max_idx}: {amp_data[max_idx]:.4f}'
    })
    
    # Stage 4: After inverse QFT
    qft_dagger = QFT(n_count, inverse=True, do_swaps=True)
    qc.append(qft_dagger, count_reg)
    sv = Statevector(qc)
    
    # Compute measurement probabilities for count register
    probs = sv.probabilities()
    # Marginalize over search register to get count register probabilities
    count_probs = np.zeros(2**n_count)
    for i in range(len(probs)):
        count_idx = i >> n_search  # Upper bits
        count_probs[count_idx] += probs[i]
    
    max_prob_idx = np.argmax(count_probs)
    M_est, theta_est = extract_M_from_measurement(max_prob_idx, n_count, N)
    
    trace_info['stages'].append({
        'name': 'After QFT†',
        'description': 'Phase converted to computational basis',
        'state_info': f'Peak at count={max_prob_idx} (binary: {format(max_prob_idx, f"0{n_count}b")})',
        'key_amplitude': f'M_estimate ≈ {M_est:.2f} (true: {M_true})',
        'theta_estimate': theta_est,
        'M_estimate': M_est,
        'count_probs': count_probs
    })
    
    return trace_info


# Trace the counting process
n_count = 4
n_search = 3
M_true = 2
oracle = create_oracle(n_search, [0, 7])

trace_data = trace_counting_state(n_count, n_search, oracle, M_true)

print("="*70)
print("QUANTUM COUNTING STATE EVOLUTION TRACE")
print("="*70)
print(f"\nProblem: N={trace_data['N']} items, M={trace_data['M_true']} solutions")
print(f"True θ = {np.degrees(trace_data['theta_true']):.2f}°")
print("\n" + "-"*70)

for i, stage in enumerate(trace_data['stages']):
    print(f"\n📍 Stage {i+1}: {stage['name']}")
    print(f"   Description: {stage['description']}")
    print(f"   State info: {stage['state_info']}")
    print(f"   Key value: {stage['key_amplitude']}")

In [ ]:
### 5.5.3 Interactive Tracer: M vs Precision Analysis

def analyze_counting_precision(n_count_values: List[int], n_search: int, 
                                M_true: int, shots: int = 1024) -> dict:
    """
    Analyze how counting precision depends on number of counting qubits.
    
    Args:
        n_count_values: List of counting qubit counts to test
        n_search: Number of search qubits
        M_true: True number of solutions
        shots: Number of simulation shots
    
    Returns:
        Analysis results
    """
    results = {'n_count': [], 'M_mean': [], 'M_std': [], 'error': []}
    N = 2**n_search
    oracle = create_oracle(n_search, list(range(M_true)))  # First M_true states as solutions
    
    # Use StatevectorSampler without backend (no qubit limit)
    sampler = SamplerV2()
    
    print(f"Precision Analysis: N={N}, M_true={M_true}")
    print("="*60)
    
    for n_count in n_count_values:
        qc = quantum_counting_circuit(n_count, n_search, oracle)
        # Decompose multiple times to break down custom gates
        qc_decomposed = qc.decompose().decompose().decompose()
        
        # Execute with SamplerV2 (statevector simulation, no backend needed)
        job = sampler.run([qc_decomposed], shots=shots)
        counts = job.result()[0].data.c.get_counts()
        
        # Extract M estimates
        M_estimates = []
        for bitstring, count in counts.items():
            measured = int(bitstring, 2)
            M_est, _ = extract_M_from_measurement(measured, n_count, N)
            M_estimates.extend([M_est] * count)
        
        M_mean = np.mean(M_estimates)
        M_std = np.std(M_estimates)
        error = abs(M_mean - M_true)
        
        results['n_count'].append(n_count)
        results['M_mean'].append(M_mean)
        results['M_std'].append(M_std)
        results['error'].append(error)
        
        print(f"t={n_count} qubits: M = {M_mean:.3f} ± {M_std:.3f}, error = {error:.3f}")
    
    return results


# Analyze precision for different counting register sizes
precision_results = analyze_counting_precision(
    n_count_values=[3, 4, 5, 6],
    n_search=3,
    M_true=2,
    shots=1024
)


In [ ]:
# Visualize precision analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: M estimate vs counting qubits
ax1 = axes[0]
ax1.errorbar(precision_results['n_count'], precision_results['M_mean'], 
             yerr=precision_results['M_std'], fmt='o-', capsize=5, linewidth=2, markersize=8)
ax1.axhline(y=2, color='r', linestyle='--', label='True M = 2')
ax1.set_xlabel('Number of counting qubits (t)', fontsize=12)
ax1.set_ylabel('Estimated M', fontsize=12)
ax1.set_title('Quantum Counting Accuracy vs Precision Qubits', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Error reduction
ax2 = axes[1]
ax2.semilogy(precision_results['n_count'], precision_results['error'], 'o-', 
             linewidth=2, markersize=8, color='orange')
ax2.set_xlabel('Number of counting qubits (t)', fontsize=12)
ax2.set_ylabel('Absolute Error |M_est - M_true|', fontsize=12)
ax2.set_title('Error Reduction with More Precision Qubits', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5.5.4 Summary: Key Relationships

| Quantity | Formula | Physical Meaning |
|----------|---------|------------------|
| **Rotation angle** | $\theta = \arcsin(\sqrt{M/N})$ | How much Grover rotates per iteration |
| **Eigenvalues** | $e^{\pm 2i\theta}$ | Grover operator's spectrum |
| **QPE output** | $\tilde{\theta} \approx 2\theta/\pi \cdot 2^t$ | Discretized phase measurement |
| **M estimate** | $N \cdot \sin^2(\pi\tilde{\theta}/2^t)$ | Inverted formula for count |
| **Precision** | $\sim 1/2^t$ | Phase resolution → M accuracy |
| **Total queries** | $O(\sqrt{N})$ | Same as standard Grover |

### 5.5.5 The "Eigenphase Encodes Solution Count" Rule

**The Rule in One Sentence**:
> "Apply QPE to the Grover operator G; the measured eigenphase ±2θ directly encodes M via sin²(θ) = M/N."

**Why This Works** (The "Magic"):

1. **Grover as a Rotation**: G rotates in the 2D subspace spanned by |s⟩ (equal superposition) and |w⟩ (target states)
2. **Eigenvalues Encode M**: The rotation angle θ is determined by the solution fraction M/N
3. **QPE Extracts θ**: Measuring the eigenphase gives us θ (up to discretization error)
4. **Inversion Recovers M**: Simple algebra: M = N · sin²(θ)

**Key Insight**: Unlike standard Grover (which finds *a* solution), quantum counting finds *how many* solutions exist—enabling:
- **Optimal iteration count**: Know exactly when to stop
- **Amplitude estimation**: Estimate success probabilities quadratically faster
- **Adaptive algorithms**: Handle unknown M gracefully

**Quantum Advantage**:
| Task | Classical | Quantum | Speedup |
|------|-----------|---------|---------|
| Count M among N | O(N) | O(√N) | Quadratic |
| Estimate probability p | O(1/ε²) | O(1/ε) | Quadratic in precision |

In [ ]:
def explore_eigenphase_M_relationship(N_values: List[int] = [8, 16, 32, 64]) -> None:
    """
    Interactive exploration of how eigenphase encodes solution count.
    
    Demonstrates the "Eigenphase Encodes Solution Count" rule:
    - G|ψ±⟩ = e^{±2iθ}|ψ±⟩  
    - sin²(θ) = M/N
    - θ = arcsin(√(M/N))
    
    Args:
        N_values: List of search space sizes to explore
    """
    print("=" * 70)
    print("EXPLORING: Eigenphase ↔ Solution Count Relationship")
    print("=" * 70)
    print("\nKey Identity: G|ψ±⟩ = e^{±2iθ}|ψ±⟩ where sin²(θ) = M/N")
    print("=" * 70)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    for ax, N in zip(axes.flatten(), N_values):
        M_range = np.arange(1, N)
        theta_values = np.arcsin(np.sqrt(M_range / N))
        eigenphase_values = 2 * theta_values  # eigenvalue = e^{±2iθ}
        
        # Optimal Grover iterations for each M
        k_optimal = np.floor(np.pi / (4 * theta_values))
        
        ax.plot(M_range, np.degrees(theta_values), 'b-', linewidth=2, label='θ (degrees)')
        ax.plot(M_range, k_optimal, 'r--', linewidth=2, label='Optimal iterations k')
        
        ax.set_xlabel('Number of solutions M', fontsize=11)
        ax.set_ylabel('Value', fontsize=11)
        ax.set_title(f'N = {N} items', fontsize=12)
        ax.legend(loc='upper right')
        ax.grid(True, alpha=0.3)
        
        # Mark special cases
        if N >= 4:
            # Mark M = N/4 (θ = 30°, k ≈ 1)
            M_quarter = N // 4
            theta_quarter = np.degrees(np.arcsin(np.sqrt(M_quarter / N)))
            ax.axvline(x=M_quarter, color='green', linestyle=':', alpha=0.7)
            ax.annotate(f'M={M_quarter}\nθ≈30°', (M_quarter, theta_quarter), 
                       textcoords="offset points", xytext=(10, 10), fontsize=9)
    
    plt.suptitle('The "Eigenphase Encodes Solution Count" Rule\n'
                 'sin²(θ) = M/N  →  Eigenvalue e^{±2iθ}', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Print specific examples
    print("\n📊 Worked Examples:")
    print("-" * 60)
    print(f"{'N':<6} {'M':<6} {'M/N':<10} {'θ (rad)':<12} {'θ (deg)':<12} {'k_opt':<8}")
    print("-" * 60)
    
    examples = [(8, 2), (16, 4), (32, 1), (64, 8)]
    for N, M in examples:
        ratio = M / N
        theta = np.arcsin(np.sqrt(ratio))
        k_opt = int(np.floor(np.pi / (4 * theta)))
        print(f"{N:<6} {M:<6} {ratio:<10.4f} {theta:<12.4f} {np.degrees(theta):<12.2f} {k_opt:<8}")
    
    print("\n🔑 Key Insight:")
    print("   The eigenphase 2θ contains ALL information about M!")
    print("   QPE extracts θ → we compute M = N·sin²(θ)")
    print("   This is why quantum counting achieves O(√N) complexity.")


# Run the exploration
explore_eigenphase_M_relationship([8, 16, 32, 64])

## Section 6: Adaptive Search (Unknown M)

### The Problem

Standard Grover needs $k = \lfloor \frac{\pi}{4}\sqrt{N/M} \rfloor$ iterations.

But what if $M$ is unknown?

### Boyer-Brassard-Høyer-Tapp Algorithm

**Strategy**: Exponentially increase iteration count until solution found.

```
m = 1
λ = 6/5  # Growth factor
while not found:
    j = random(0, m-1)
    result = grover_with_j_iterations()
    if is_solution(result):
        return result
    m = min(λ * m, √N)
```

In [ ]:
def adaptive_grover_search(n_qubits: int, oracle_func: Callable, 
                           is_solution_func: Callable,
                           max_queries: int = None) -> Tuple[str, int]:
    """
    Adaptive Grover search when M is unknown.
    
    Args:
        n_qubits: Number of qubits
        oracle_func: Function returning oracle circuit
        is_solution_func: Function checking if result is valid
        max_queries: Maximum oracle queries allowed
    
    Returns:
        (solution_bitstring, total_queries)
    """
    N = 2**n_qubits
    if max_queries is None:
        max_queries = int(10 * np.sqrt(N))
    
    # Setup backend and sampler for transpiled circuits
    backend = FakeManilaV2()
    sampler = SamplerV2()
    λ = 6/5  # Growth factor (optimal for unknown M)
    m = 1
    total_queries = 0
    
    print(f"Adaptive Grover Search: N = {N}")
    print("-" * 50)
    
    while total_queries < max_queries:
        # Pick random number of iterations from [0, m-1]
        j = random.randint(0, int(m) - 1) if int(m) > 0 else 0
        
        # Build and run Grover circuit with j iterations
        oracle = oracle_func()
        qc = QuantumCircuit(n_qubits, n_qubits)
        qc.h(range(n_qubits))
        
        G = create_grover_operator(n_qubits, oracle)
        for _ in range(j):
            qc.compose(G, inplace=True)
        
        qc.measure(range(n_qubits), range(n_qubits))
        
        # Execute with SamplerV2 and transpiled circuit using fake backend
        transpiled_qc = transpile(qc, backend=backend, optimization_level=3)
        job = sampler.run([transpiled_qc], shots=1)
        result = list(job.result()[0].data.c.get_counts().keys())[0]
        
        total_queries += j + 1  # j iterations + 1 for measurement
        
        print(f"  m={m:.1f}, j={j} iterations: measured |{result}⟩", end="")
        
        # Check if solution found
        if is_solution_func(result):
            print(" ✓ FOUND!")
            return result, total_queries
        else:
            print(" ✗")
        
        # Increase range
        m = min(λ * m, np.sqrt(N))
    
    return None, total_queries


# Test adaptive search
n = 4  # N = 16 items
solutions = [5, 10]  # M = 2 solutions

def make_oracle():
    return create_oracle(n, solutions)

def check_solution(bitstring):
    return int(bitstring, 2) in solutions

print(f"\nSearching for solutions {solutions} among N={2**n} items")
print(f"(M={len(solutions)} solutions, unknown to algorithm)\n")

result, queries = adaptive_grover_search(n, make_oracle, check_solution)

print(f"\nResult: Found |{result}⟩ = {int(result, 2)} using {queries} queries")
print(f"Theoretical optimal (if M known): {int(np.pi/4 * np.sqrt(2**n / len(solutions)))} queries")

## Section 7: SAT Oracle Construction

### From Boolean Formula to Phase Oracle

Given CNF formula: $(x_1 \lor \neg x_2) \land (\neg x_1 \lor x_3) \land (x_2 \lor x_3)$

**Goal**: Build oracle that flips phase of satisfying assignments.

In [ ]:
def create_sat_oracle(n_vars: int, clauses: List[List[int]]) -> QuantumCircuit:
    """
    Create phase oracle for SAT problem in CNF form.
    
    Args:
        n_vars: Number of boolean variables
        clauses: List of clauses, each clause is a list of literals.
                 Positive integer = variable (1-indexed)
                 Negative integer = negated variable
                 Example: [[1, -2], [-1, 3]] means (x₁ ∨ ¬x₂) ∧ (¬x₁ ∨ x₃)
    
    Returns:
        Oracle circuit
    """
    n_clauses = len(clauses)
    # Need ancilla qubits: one per clause + one output
    n_ancilla = n_clauses + 1
    n_total = n_vars + n_ancilla
    
    qc = QuantumCircuit(n_total, name='SAT_Oracle')
    
    clause_ancillas = list(range(n_vars, n_vars + n_clauses))
    output_ancilla = n_vars + n_clauses
    
    # For each clause, compute if it's UNSATISFIED into ancilla
    for c_idx, clause in enumerate(clauses):
        clause_qubits = []
        flip_back = []
        
        for literal in clause:
            var_idx = abs(literal) - 1  # Convert to 0-indexed
            clause_qubits.append(var_idx)
            
            # If literal is negative (¬x), flip to check for 0
            # If literal is positive (x), flip to check for 0 (we detect unsatisfied)
            if literal > 0:
                qc.x(var_idx)
                flip_back.append(var_idx)
        
        # Clause is UNSATISFIED if all literals are 0
        # Multi-controlled X: if all clause qubits are |1⟩ → flip ancilla
        if len(clause_qubits) == 1:
            qc.cx(clause_qubits[0], clause_ancillas[c_idx])
        else:
            qc.mcx(clause_qubits, clause_ancillas[c_idx])
        
        # Undo X gates
        for var_idx in flip_back:
            qc.x(var_idx)
    
    # If ALL clause ancillas are |0⟩ (all satisfied), flip output
    # First, flip all clause ancillas (so we detect all |0⟩)
    qc.x(clause_ancillas)
    
    # Multi-controlled Z on output (phase flip if satisfiable)
    qc.h(output_ancilla)
    if len(clause_ancillas) == 1:
        qc.cx(clause_ancillas[0], output_ancilla)
    else:
        qc.mcx(clause_ancillas, output_ancilla)
    qc.h(output_ancilla)
    
    # Undo X on clause ancillas
    qc.x(clause_ancillas)
    
    # Uncompute clause ancillas (reverse the clause computations)
    for c_idx in range(n_clauses - 1, -1, -1):
        clause = clauses[c_idx]
        clause_qubits = []
        flip_back = []
        
        for literal in clause:
            var_idx = abs(literal) - 1
            clause_qubits.append(var_idx)
            if literal > 0:
                qc.x(var_idx)
                flip_back.append(var_idx)
        
        if len(clause_qubits) == 1:
            qc.cx(clause_qubits[0], clause_ancillas[c_idx])
        else:
            qc.mcx(clause_qubits, clause_ancillas[c_idx])
        
        for var_idx in flip_back:
            qc.x(var_idx)
    
    return qc


# Example: (x₁ ∨ ¬x₂) ∧ (¬x₁ ∨ x₃) ∧ (x₂ ∨ x₃)
# Variables: x₁=q0, x₂=q1, x₃=q2
clauses = [
    [1, -2],   # x₁ ∨ ¬x₂
    [-1, 3],   # ¬x₁ ∨ x₃
    [2, 3]     # x₂ ∨ x₃
]

sat_oracle = create_sat_oracle(3, clauses)
print("SAT Oracle for (x₁ ∨ ¬x₂) ∧ (¬x₁ ∨ x₃) ∧ (x₂ ∨ x₃):")
print(f"Total qubits: 3 vars + {len(clauses)} clause ancillas + 1 output = {sat_oracle.num_qubits}")
print(sat_oracle.draw(fold=100))

In [ ]:
# Verify SAT oracle marks correct states
def evaluate_cnf(assignment: str, clauses: List[List[int]]) -> bool:
    """Classically evaluate CNF formula."""
    values = {i+1: int(b) for i, b in enumerate(reversed(assignment))}
    
    for clause in clauses:
        clause_satisfied = False
        for literal in clause:
            var = abs(literal)
            val = values[var]
            if literal > 0 and val == 1:
                clause_satisfied = True
            elif literal < 0 and val == 0:
                clause_satisfied = True
        if not clause_satisfied:
            return False
    return True


print("Verification of SAT Oracle:")
print("Formula: (x₁ ∨ ¬x₂) ∧ (¬x₁ ∨ x₃) ∧ (x₂ ∨ x₃)")
print("\n| x₃x₂x₁ | Classical | Oracle Phase |")
print("|--------|-----------|--------------|")

n_vars = 3
n_ancilla = len(clauses) + 1

for i in range(8):
    assignment = format(i, '03b')  # x₃x₂x₁
    classical_result = evaluate_cnf(assignment, clauses)
    
    # Build test circuit
    qc_test = QuantumCircuit(n_vars + n_ancilla)
    
    # Prepare variable state
    for j, bit in enumerate(reversed(assignment)):
        if bit == '1':
            qc_test.x(j)
    
    # Apply oracle
    qc_test.compose(sat_oracle, inplace=True)
    
    # Get statevector
    sv = Statevector(qc_test)
    
    # Find amplitude of our prepared state (ancillas should be |0⟩)
    state_idx = i  # First 3 qubits, ancillas all 0
    amplitude = sv.data[state_idx]
    phase = "−1" if np.real(amplitude) < 0 else "+1"
    
    sat_str = "SAT" if classical_result else "UNSAT"
    mark = "✓" if (classical_result and phase == "−1") or (not classical_result and phase == "+1") else "✗"
    
    print(f"| {assignment}    | {sat_str:5s}     | {phase}           | {mark}")

print("\nSatisfying assignments marked with −1 phase: 011, 101, 110, 111")

## Section 8: Exercises

### 🟢 Exercise 1: Basic Quantum Counting

Implement quantum counting for N=16, M=4 and verify the result.

In [ ]:
# Exercise 1: Your implementation here

# TODO:
# 1. Create oracle for 4 solutions among 16 items
# 2. Build quantum counting circuit with 6 precision qubits
# 3. Run and verify M ≈ 4

# Your code here:
# n_search = 4  # N = 16
# n_count = 6   # 6 bits precision
# M_true = 4
# ...

### 🟡 Exercise 2: Precision Analysis

Plot the relationship between counting qubits and M estimation error for a fixed problem.

In [ ]:
# Exercise 2: Your implementation here

# TODO:
# 1. Fix n_search=4, M=3
# 2. Vary n_count from 3 to 8
# 3. Plot error vs n_count
# 4. Fit error ∝ 1/2^n_count

# Your code here:
# ...

### 🔴 Exercise 3: SAT Solver with Grover

Solve the following SAT problem using Grover's algorithm:

$(x_1 \lor x_2 \lor \neg x_3) \land (\neg x_1 \lor x_3) \land (x_2 \lor x_3 \lor x_4) \land (\neg x_2 \lor \neg x_4)$

In [ ]:
# Exercise 3: Your implementation here

# TODO:
# 1. Build SAT oracle for the given formula
# 2. First use quantum counting to estimate M
# 3. Then use optimal Grover iterations to find a solution
# 4. Verify the solution classically

# clauses = [
#     [1, 2, -3],    # x₁ ∨ x₂ ∨ ¬x₃
#     [-1, 3],       # ¬x₁ ∨ x₃
#     [2, 3, 4],     # x₂ ∨ x₃ ∨ x₄
#     [-2, -4]       # ¬x₂ ∨ ¬x₄
# ]

# Your code here:
# ...

## Section 9: Common Traps and Solutions

### Trap 1: Phase Ambiguity in Counting

QPE can measure $\theta$ or $-\theta$ (mod $2\pi$). Both give same $M$ because $\sin^2(\theta) = \sin^2(-\theta)$.

### Trap 2: Ancilla Cleanup in SAT Oracle

❌ **Wrong**: Forget to uncompute ancillas
```python
# Compute clause values
# Apply phase flip
# Done! (ancillas left dirty)
```

✅ **Correct**: Mirror computation to clean up
```python
# Compute clause values
# Apply phase flip  
# Uncompute clause values (reverse order!)
```

### Trap 3: Wrong Iteration Count with Estimated M

If $M$ estimate has error, optimal iterations may overshoot.

✅ **Solution**: Use fewer iterations or randomize.

## Section 10: Summary

### Key Takeaways

1. **Quantum Counting**: QPE on Grover operator extracts $M$ from eigenphase
2. **Amplitude Estimation**: Quadratic speedup over Monte Carlo ($O(1/\epsilon)$ vs $O(1/\epsilon^2)$)
3. **Adaptive Search**: Exponential backoff finds solution in $O(\sqrt{N/M})$ even when $M$ unknown
4. **SAT Solving**: Oracle construction from CNF formula enables Grover for NP problems

### Complexity Summary

| Application | Classical | Quantum |
|-------------|-----------|----------|
| Count solutions | $O(N)$ | $O(\sqrt{N})$ |
| Estimate probability | $O(1/\epsilon^2)$ | $O(1/\epsilon)$ |
| Search (M unknown) | $O(N)$ | $O(\sqrt{N/M})$ |
| SAT solving | $O(2^n)$ | $O(2^{n/2})$ |

## 📚 References

1. Brassard, Høyer, Mosca, Tapp (2002). "Quantum Amplitude Amplification and Estimation"
2. Boyer, Brassard, Høyer, Tapp (1998). "Tight Bounds on Quantum Searching"
3. Nielsen & Chuang, Chapter 6.4: "Quantum Counting"

---

**Next Module**: [Module 7.10: QAOA Advanced](Module-10-QAOA-Advanced.md)